# PetFinder.my Adoption Prediction
---

## 資料

- [Kaggleにおける「特徴量エンジニアリング」の位置づけ　〜『機械学習のための特徴量エンジニアリング』に寄せて〜 - u++の備忘録](https://upura.hatenablog.com/entry/2019/02/22/221125)
    - kaggle勉強会第二回のまえにこれに目を通しておく

## 前提

- kernels only competition
- `submission.csv` という名前で提出
    - (PetID, AdoptionSpeed)の２つ組
- 採点はquadratic weighted kappaをベースに行われる
- 予測結果は0-4のratingsを返すようにする
    - 0-4の意味はデータセットの方を参照
- 3/28 11:59(UTC) が最終提出期限
- CPUだけなら6h, GPU使用なら2h分submit時のkernel実行に使用できる
- 使用できるデータ
    - [https://www.kaggle.com/c/petfinder-adoption-prediction/data](https://www.kaggle.com/c/petfinder-adoption-prediction/data)
        - 各種データフィールドの概要はこちらで確認
    - 他外部からアップロードしてopen datasetとして登録したもの

## initalization

各種必要なlibraryのimportと初期設定

- numpy
    - [http://www.numpy.org/](http://www.numpy.org/)
    - 数値計算を効率的に行うやつ
    - 今回は予測結果のargmaxぐらいしか使っていない
- pandas
    - [https://pandas.pydata.org/](https://pandas.pydata.org/)
    - データ分析用
    - train, testデータの読み込みとPandasDataFrameを使用
- seaborn
    - plot用library
    - matplotlibより見た目きれいらしいので試してみる
- matplotlib
    - [https://matplotlib.org/](https://matplotlib.org/)
    - 2Dのplotting library
    - 従属変数の重要度の可視化に使用
    - inlineとは
        - jupyter notebook上でmatplotlibのグラフをplotする
        - これがないと表示されない
- warings
    - [https://docs.python.org/ja/3/library/warnings.html](https://docs.python.org/ja/3/library/warnings.html)
    - filterwarningsで一部警告を無視
- kaggle上で使用できるlibraries
    - [https://github.com/kaggle/docker-python](https://github.com/kaggle/docker-python)
    - ここでinstallされてるやつ
    - jupyter上で `!pip list` でも確認できる

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

## kaggleのKernelで使用できるlibrary一覧

In [ ]:
!pip list

## trainデータの確認と読み込み

他のkernelのsampleだとpythonコードでdirectoryのfile一覧を表示しているが、 `ls` でもよい

In [ ]:
!ls -l ../input

pandasを使用して学習データを読み込み、行列のサイズと内容を確認する

In [ ]:
df = pd.read_csv('../input/train/train.csv')
print (df.shape)
df.head()

学習データの列の情報を確認する。ここでは名前と型を確認し、それぞれのデータフィールドの概要は下記で確認する
https://www.kaggle.com/c/petfinder-adoption-prediction/data

In [ ]:
df.info()

trainデータには目的変数である `AdaptionSpeed` が含まれているので、その分布を確認してみる

In [ ]:
sns.countplot(df.AdoptionSpeed)

## ベースモデルの作成

まずは単純なモデルをつくる
扱いにくい数値以外の変数を落として、そのデータによってモデルを学習する

まずは学習データを従属変数と目的変数に分ける

In [ ]:
X = df.drop('AdoptionSpeed', axis=1)
y = df['AdoptionSpeed']

まずは単純に学習させたいので、数値型だけ残す

In [ ]:
X = X.select_dtypes(exclude=[object])
print (X.shape)
X.head()

上記でintとfloatの列だけ残したので、これを使ってモデルを学習する
このときに下記のlibrariesを使用する

- lightgbm
    - https://lightgbm.readthedocs.io/en/latest/
    - 今回使用する予測器
    - Gradient Boosting
- sklearn.model_selection.train_test_split
    - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
    - Validationのために、学習データを更に学習データとテストデータに分割するのに使用
- sklearn.metrics.cohen_kappa_score
    - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.cohen_kappa_score.html
    - 今回の評価がquadratic weighted kappaなので、これで出力した予測結果の評価を行う

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score

- test_sizeは雑に半分ずつ
    - 50% train
    - 25% eval
    - 25% test
- random_stateの42の元ネタ
    - [生命、宇宙、そして万物についての究極の疑問の答え - Wikipedia](https://ja.wikipedia.org/wiki/%E7%94%9F%E5%91%BD%E3%80%81%E5%AE%87%E5%AE%99%E3%80%81%E3%81%9D%E3%81%97%E3%81%A6%E4%B8%87%E7%89%A9%E3%81%AB%E3%81%A4%E3%81%84%E3%81%A6%E3%81%AE%E7%A9%B6%E6%A5%B5%E3%81%AE%E7%96%91%E5%95%8F%E3%81%AE%E7%AD%94%E3%81%88)
- Q
    - lgb.Dataset
        - categorical_feature
    - lgb.train
        - 各parameter
    - train,eval, testの意味

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.5, random_state=42)
X_eval, X_test, y_eval, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=42)

f_names = X.columns.tolist()
cat_feature = list(set(f_names) - set(['Age', 'MaturitySize', 'FurLength', 'Fee']))

train_data = lgb.Dataset(X_train, label=y_train, feature_name=f_names, categorical_feature=cat_feature)
train_eval = lgb.Dataset(X_eval, label=y_eval, reference=train_data)

params = {'num_leaves': 128,
         'objective': 'multiclass',
         'max_depth': -1,
         'learning_rate': 0.1,
         "boosting": "gbdt",
         "metric": 'multi_logloss',
         "num_boost_round": 2000,
         "early_stopping_rounds": 20,
         "feature_fraction": 0.9,
         "bagging_freq": 5,
         "bagging_fraction": 0.8,
         "bagging_seed": 11,
        #  "lambda_l1": 0.1,
         # "lambda_l2": 0.1,
         "random_state": 42,          
         "verbosity": -1,
         "num_class": 5
}

clf = lgb.train(params, train_data, valid_sets=[train_data, train_eval])

## テストデータでの予測とスコアの確認

上記で学習したので、実際に予測して評価結果を確認してみる

`predict` にテストデータを入れて予測結果を取得する
`num_iteration`を省略したとき、 `best_iteration`が使える場合それを使用する
結果はnumpyのarrayで取得でき、各行に5つの予測値が入っている。それぞれPetIDに対してAdaptionSpeedの予測値となる

https://lightgbm.readthedocs.io/en/latest/Python-API.html#lightgbm.Booster.predict


In [ ]:
pred = clf.predict(X_test)

この予測結果を `cohen_kappa_score` に入れて評価を確認してみる
今回は評価のところで `quadratic weighted kappa` とあるので、weightsには `quadratic` を指定する
https://www.kaggle.com/c/petfinder-adoption-prediction#evaluation

In [ ]:
print (pred)
cohen_kappa_score(pred.argmax(1), y_test, weights='quadratic')

結果がでたので、目的変数の分布をざっと確認して、もとの学習データとの分布と大きく傾向が異ならないか確認してみる

In [ ]:
sns.countplot(pred.argmax(1))

lgbの `plot_importance` を使って従属変数のうち予測するうえで重要な変数を確認する
この内容は `matplotlib.pyplot` を使って図で確認してみる
`Age` が突出しているけど、特にこのあと何もしない

In [ ]:
lgb.plot_importance(clf, max_num_features=20)
plt.show()

## 定義したモデルで全データを使い再学習し、テストデータの予測をする

上記で予測できそうなことは確認できたので分割していた全データを使って再度学習し直す
test_sizeは20%にする

In [ ]:
params = {'num_leaves': 128,
         'objective': 'multiclass',
         'max_depth': -1,
         'learning_rate': 0.1,
         "boosting": "gbdt",
         "metric": 'multi_logloss',
         "num_boost_round": 2000,
         "early_stopping_rounds": 20,
         "feature_fraction": 0.9,
         "bagging_freq": 5,
         "bagging_fraction": 0.8,
         "bagging_seed": 11,
        #  "lambda_l1": 0.1,
         # "lambda_l2": 0.1,
         "random_state": 42,          
         "verbosity": -1,
         "num_class": 5
}

X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=42)

all_train_data = lgb.Dataset(X_train, label=y_train, feature_name=f_names, categorical_feature=cat_feature)
all_train_eval = lgb.Dataset(X_eval, label=y_eval, reference=train_data)

sub_clf = lgb.train(params, all_train_data, valid_sets=[all_train_data, all_train_eval])

上記のモデルを使用して予測結果を出力する
テストデータは以下のように読み込む

In [ ]:
test_df = pd.read_csv('../input/test/test.csv')

今回は数値以外を欠落した状態で学習しているので、テストデータも同じ次元に揃える
`shape` を使って列数を確認する

In [ ]:
test_df_only_nums = test_df.select_dtypes(exclude=[object])

print ('学習データ：', X.shape)
print ('テストデータ：', test_df_only_nums.shape)

今回はそれぞれのAdaptionSpeedの予測値で一番可能性の高いものを予測結果として返す
AdaptionSpeedは `0-4` の値で表すので、numpyの `argmax` で一番値が大きいもののindexを返すようにする

In [ ]:
prediction = sub_clf.predict(test_df_only_nums).argmax(1)

予測したので、結果の分布を先程同様に確認してみる

一応それっぽく結果が出ているようだ

In [ ]:
sns.countplot(prediction)

## 結果の提出

予測した結果が問題なさそうなので、提出用のCSVを出力する

結果用のpandas DataFrameを用意し、予測結果を記録する
今回は(PetID, AdoptionSpeed)の２つ組を記録したCSVを出力する必要があるので、それに揃える

DataFrameの列は揃えておいたので、 `to_csv` を使って指定されたファイル名である `submission.csv` で出力する
列の名前 (PetIDとAdoptionSpeed)はCSVに不要なので、 `index=False` で出力しないようにする

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html


In [ ]:
sub_df = pd.DataFrame()
sub_df['PetID'] = test_df['PetID']
sub_df['AdoptionSpeed'] = prediction
sub_df.head()

In [ ]:
sub_df.to_csv('submission.csv', index=False)